<a href="https://colab.research.google.com/github/mace1341/studious-invention/blob/main/FantasyFootball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!git clone https://github.com/cwendt94/espn-api.git
%cd espn-api
!pip install -r requirements.txt


In [ ]:
from espn_api.football import League
import pandas as pd

# Parameters (migrate to secrets file)
myleague = 2052062736
yr = 2024
my_imrid = '7f021560-620b-11ef-9905-f706e5643846'
my_swid = '{AD57828E-390B-4EE8-BB96-00D430E4F8D0}'
my_espn_s2 = 'AEBhxBJnI162GdSL4xMmm5HMaz9KcWasgvbaH0bFf4mxaQwD3o8twaicsMrq2sG0J78NNBxeEaBQtL68SSLSBxRKQh8Av1lx6paGCNXSMyh9df%2FshtOy85IwIWva9wBw8vafAm6XFI3MuZ1gQq31jq58vU13MhzaXJMbsJiFa3JutxXvst6SHoVCO3o6ur4YWyndqpM7DuUu8nXVzWMtcMjP6LQfA%2FdmcsVVr307Kz7dFtyNj14mKJO2iSKXhvIeyHVqCkS0UyaG9CS6tMp%2Ff5ij'



In [ ]:
league = League(league_id=myleague,
                year=yr,
                espn_s2 = my_espn_s2,
                swid = my_swid)


In [ ]:
# Create draft data set
draft_dict = {'name':[], 'position':[], 'proTeam':[], 'proj':[]}

# Populate
for player in league.free_agents(size=400):
    draft_dict['name'].append(player.name)
    draft_dict['position'].append(player.position)
    draft_dict['proTeam'].append(player.proTeam)
    draft_dict['proj'].append(player.projected_points)

# Create dataframe
draft_df = pd.DataFrame(draft_dict)
draft_df.sort_values(by=['proj'], ascending=False, inplace=True)


In [ ]:
# Analyze Draft

# Function to calculate quartiles by position
def calculate_quartiles_by_position(df):
    quartiles = df.groupby('position')['proj'].quantile([0.25, 0.5, 0.75]).unstack()
    quartiles.columns = ['Q1', 'Median', 'Q3']  # Rename columns for clarity

    # Calculate distinct counts by position
    counts = df.groupby('position')['name'].nunique().reset_index()
    counts.columns = ['position', 'Player Count']

    # Merge quartiles and counts
    result = pd.merge(quartiles, counts, on='position')

    return result

# Calculate quartiles by position
draft_df2 = calculate_quartiles_by_position(draft_df)

display(draft_df2)


,position,Q1,Median,Q3,Player Count
0,D/ST,78.7475,93.820,116.0075,32
1,K,126.3650,132.510,138.0750,35
2,QB,55.2850,247.570,280.1950,47
3,RB,43.3725,86.665,156.8525,104
4,TE,56.3850,75.830,111.7075,48
5,WR,57.0800,111.300,163.4900,134


In [ ]:
league.settings.position_slot_counts

{'QB': 1,
 'TQB': 0,
 'RB': 2,
 'RB/WR': 0,
 'WR': 3,
 'WR/TE': 0,
 'TE': 1,
 'OP': 0,
 'DT': 0,
 'DE': 0,
 'LB': 0,
 'DL': 0,
 'CB': 0,
 'S': 0,
 'DB': 0,
 'DP': 0,
 'D/ST': 1,
 'K': 1,
 'P': 0,
 'HC': 0,
 'BE': 8,
 'IR': 1,
 '': 0,
 'RB/WR/TE': 1,
 'ER': 0}

In [ ]:
import numpy as np

a = [1, 2, 3, 4, 5, 6, 7, 8]
b = np.random.choice(a, size=1, replace=False)
c = np.setdiff1d(a, b)

plyrtest = np.array(draft_df['name'])
poscond = (draft_df['position'] == 'QB')
postest = np.array(draft_df[poscond]['name'])




In [ ]:
# simulated draft
roster = []
pool = postest

# selection
pick = np.random.choice(pool, size=1, replace=False)
roster.append(pick.item())
pool2 = np.setdiff1d(pool, pick)



In [ ]:
display(draft_df)

,name,position,proTeam,proj
14,Patrick Mahomes,QB,KC,340.62
11,Josh Allen,QB,BUF,336.54
12,Jalen Hurts,QB,PHI,335.00
18,Lamar Jackson,QB,BAL,325.84
58,Joe Burrow,QB,CIN,305.87
...,...,...,...,...
249,J.J. McCarthy,QB,MIN,0.00
240,Darren Waller,TE,NYG,0.00
315,Louis Rees-Zammit,RB,KC,0.00
375,JuJu Smith-Schuster,WR,None,0.00


In [ ]:
# How to look up the max proj
max_proj_index = draft_df['proj'].idxmax()
max_proj_player = draft_df.loc[max_proj_index]

# How to invert a sequence
draft_order = np.arange(10)
reverse_order = draft_order[::-1]
full_order = np.tile(np.append(draft_order, reverse_order),9)

# Position order
print(np.random.shuffle(np.append(np.repeat('RB',4), np.repeat('WR',6))))


None


array([2, 9, 1, 6, 0, 3, 5, 8, 4, 7])

In [ ]:
# Dropping a row from a dataframe
df_qb = draft_df[draft_df['position'] == 'QB'].copy()
df_qb.drop(index=14, inplace=True)
display(df_qb)

In [ ]:
# Pool Filters
cond_Q = (draft_df['position'] == 'QB')
cond_QRW = ((draft_df['position'] == 'QB')|(draft_df['position'] == 'RB')|(draft_df['position'] == 'WR'))
cond_RW = ((draft_df['position'] == 'RB')|(draft_df['position'] == 'WR'))
cond_R = (draft_df['position'] == 'RB')
cond_W = (draft_df['position'] == 'WR')
cond_RWT = ((draft_df['position'] == 'RB')|(draft_df['position'] == 'WR')|(draft_df['position'] == 'TE'))
cond_TKD = ((draft_df['position'] == 'TE')|(draft_df['position'] == 'K')|(draft_df['position'] == 'D/ST'))
cond_T = (draft_df['position'] == 'TE')
cond_K = (draft_df['position'] == 'K')
cond_D = (draft_df['position'] == 'D/ST')

cond = {'Q': cond_Q, 'QRW': cond_QRW, 'RW': cond_RW, 'R': cond_R,
        'W': cond_W, 'RWT': cond_RWT, 'TKD': cond_TKD, 'T': cond_T,
        'K': cond_K, 'D': cond_D}

# How to look up the max proj with conditions
max_proj_index2 = draft_df[cond['R']]['proj'].idxmax()
max_proj_player2 = draft_df.loc[max_proj_index2]
print(max_proj_player2)

name        Christian McCaffrey
position                     RB
proTeam                      SF
proj                     301.42
Name: 5, dtype: object


In [ ]:
arr = np.arange(5)
np.random.shuffle(arr)
